# Boost Switching regulator LM3478
- Supply: 15 V
- Output voltage: Between 20 and 40 V (theoretically up to 45 V)
	- Make the output voltage variable through some variable resistor (Manually / voltage controlled)

## Switching architecture
- Switching frequency: 100 kHz - 1 MHz
- PWM, current mode control architecture
- Only fits for low-side switching (no level shifting)

## Other features
- Thermal shutdown
- Short-cicuit protection
- Over-voltage protection
- **Internal soft-start**

# LTSpice Simulation
## Issue with simulation
- Vds gets to a max between 3 and 4 V (since Vcc is 3.3 V)
  - Vgs(th) is 2-4 V
  - It seems like the Vgs is simply somewhat too small for the fet to be in the linear conducting region
## Isense
- Current sensing pin. Should take the voltage drop generated into it.